In [1]:
import json
import numpy as np
import pandas as pd
from dateutil import parser
import preprocess_hop_log, merge_hop_tick, aggregate_rows_by_time
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [ ]:
# path_hoplog = '../raw_data/hopLogger/12/'
# path_ticklog = '../raw_data/tickLogger/12/'
# path_mergeddf = '../preprocessed_data/12/'

DIR_NAME = 210

path_hoplog = '../raw_data/hopLogger/' + str(DIR_NAME) + '/'
path_ticklog = '../raw_data/tickLogger/' + str(DIR_NAME) + '/'
# path_hoplog = '../../wowsan/log/hopLogger/' + str(DIR_NAME) + '/'
# path_ticklog = '../../wowsan/log/tickLogger/' + str(DIR_NAME) + '/'
path_mergeddf = '../preprocessed_data_sttransformer/' + str(DIR_NAME) + '/'


In [3]:
NUM_BROKERS = 5
BROKER_PORT = 50003

## Hop Log 전처리

In [4]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/hopLogger/116/'
data = []
with open(path_hoplog + f'{BROKER_PORT}.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
hop_df_raw = pd.DataFrame(data)
type(hop_df_raw)

pandas.core.frame.DataFrame

In [5]:
hop_df_raw

,Node,PerformanceInfo,level,msg,time
0,localhost:50003,"[{'BrokerId': 'localhost:50001', 'Cpu': '64.282521', 'Memory': '22675456', 'QueueLength': '0', 'QueueTime': '0.303324', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '0.303324', 'InterArrivalTime': '0.000000', 'Throughput': '3.296801', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:52.7619299 +0900 KST m=+5.818396501'}, {'BrokerId': 'localhost:50002', 'Cpu': '53.477604', 'Memory': '23056384', 'QueueLength': '0', 'QueueTime': '0.311918', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '0.311918', 'InterArrivalTime': '0.000000', 'Throughput': '3.205973', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:53.3877958 +0900 KST m=+6.508136701'}, {'BrokerId': 'localhost:50003', 'Cpu': '71.543500', 'Memory': '22896640', 'QueueLength': '0', 'QueueTime': '0.308376', 'AverageQueueTime': '0.308376', 'ServiceTime': '0.000000', 'AverageServiceTime': '0.000000', 'ResponseTime': '0.308376', 'InterArrivalTime': '0.000000', 'Throughput': '3.242796', 'AverageThroughput': '3.242796', 'Timestamp': '2024-11-30 17:35:54.0443138 +0900 KST m=+7.150063301'}]",info,Advertisement(apple > 0.25195215418436445) 194f50a8-90e8-45b7-8c3c-bed7fb62e514,2024-11-30 17:35:54
1,localhost:50003,"[{'BrokerId': 'localhost:50004', 'Cpu': '66.722926', 'Memory': '23662592', 'QueueLength': '0', 'QueueTime': '0.100927', 'AverageQueueTime': '', 'ServiceTime': '0.493589', 'AverageServiceTime': '', 'ResponseTime': '0.594516', 'InterArrivalTime': '0.559047', 'Throughput': '1.682040', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:55.5773986 +0900 KST m=+8.676233501'}, {'BrokerId': 'localhost:50003', 'Cpu': '71.700718', 'Memory': '23171072', 'QueueLength': '0', 'QueueTime': '0.303256', 'AverageQueueTime': '0.305816', 'ServiceTime': '0.622440', 'AverageServiceTime': '0.622440', 'ResponseTime': '0.925696', 'InterArrivalTime': '1.280409', 'Throughput': '1.080268', 'AverageThroughput': '1.077289', 'Timestamp': '2024-11-30 17:35:56.6000112 +0900 KST m=+9.705760701'}]",info,Advertisement(apple > 0.8905283893938075) cd793b01-1cf5-4817-98b8-2f6804b7d137,2024-11-30 17:35:56
2,localhost:50003,"[{'BrokerId': 'localhost:50005', 'Cpu': '67.067395', 'Memory': '22700032', 'QueueLength': '0', 'QueueTime': '0.126521', 'AverageQueueTime': '', 'ServiceTime': '0.141435', 'AverageServiceTime': '', 'ResponseTime': '0.267956', 'InterArrivalTime': '0.944930', 'Throughput': '3.731953', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:57.813104 +0900 KST m=+10.936167001'}, {'BrokerId': 'localhost:50004', 'Cpu': '69.896775', 'Memory': '24137728', 'QueueLength': '0', 'QueueTime': '0.113377', 'AverageQueueTime': '', 'ServiceTime': '0.115075', 'AverageServiceTime': '', 'ResponseTime': '0.228452', 'InterArrivalTime': '0.971101', 'Throughput': '4.377291', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:58.3561582 +0900 KST m=+11.454993101'}, {'BrokerId': 'localhost:50003', 'Cpu': '70.441196', 'Memory': '23486464', 'QueueLength': '0', 'QueueTime': '0.329153', 'AverageQueueTime': '0.313595', 'ServiceTime': '0.723655', 'AverageServiceTime': '0.673048', 'ResponseTime': '1.052808', 'InterArrivalTime': '1.671994', 'Throughput': '0.949840', 'AverageThroughput': '1.013538', 'Timestamp': '2024-11-30 17:35:59.0810738 +0900 KST m=+12.186823301'}]",info,Advertisement(apple > 0.9865251742875035) 4c923870-0d6b-4411-b9be-0ccdd3e6031d,2024-11-30 17:35:59
3,localhost:50003,"[{'BrokerId': 'localhost:50004', 'Cpu': '67.996619', 'Memory': '24391680', 'QueueLength': '0', 'QueueTime': '0.141062', 'AverageQueueTime': '', 'ServiceTime': '0.518791', 'AverageServiceTime': '', 'ResponseTime': '0.659852', 'InterArrivalTime': '1.207823', 'Throughput': '1.515490', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:36:00.5385549 +0900 KST m=+13.637389801'}, {'BrokerId': 'localhost:50003', 'Cpu': '64.739492', 'Memory': '23744512', 'QueueLength': '0', 'Queu

In [6]:
# hop_df_raw 타입 확인
hop_df_raw.dtypes

Node               object
PerformanceInfo    object
level              object
msg                object
time               object
dtype: object

In [7]:
hop_df = preprocess_hop_log.parse_performance_info(NUM_BROKERS, hop_df_raw)

In [8]:
hop_df

,Node,level,msg,time,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_Cpu,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_InterArrivalTime,B3_Throughput,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,localhost:50003,info,Advertisement(apple > 0.25195215418436445) 194f50a8-90e8-45b7-8c3c-bed7fb62e514,2024-11-30 17:35:54,64.282521,22675456.0,0.0,0.303324,0.000000,0.303324,0.000000,3.296801,5.818397,NaN,53.477604,23056384.0,0.0,0.311918,0.000000,0.311918,0.000000,3.205973,6.508137,NaN,71.543500,22896640.0,0.0,0.308376,0.000000,0.308376,0.000000,3.242796,7.150063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,localhost:50003,info,Advertisement(apple > 0.8905283893938075) cd793b01-1cf5-4817-98b8-2f6804b7d137,2024-11-30 17:35:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.700718,23171072.0,0.0,0.303256,0.622440,0.925696,1.280409,1.080268,9.705761,NaN,66.722926,23662592.0,0.0,0.100927,0.493589,0.594516,0.559047,1.682040,8.676234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,localhost:50003,info,Advertisement(apple > 0.9865251742875035) 4c923870-0d6b-4411-b9be-0ccdd3e6031d,2024-11-30 17:35:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.441196,23486464.0,0.0,0.329153,0.723655,1.052808,1.671994,0.949840,12.186823,NaN,69.896775,24137728.0,0.0,0.113377,0.115075,0.228452,0.971101,4.377291,11.454993,NaN,67.067395,22700032.0,0.0,0.126521,0.141435,0.267956,0.94493,3.731953,10.936167,NaN
3,localhost:50003,info,Advertisement(apple > 0.6275339485574908) c5c491be-e864-4cc9-a729-803843b160ac,2024-11-30 17:36:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.739492,23744512.0,0.0,0.326061,0.820683,1.146745,1.837630,0.872034,14.518270,NaN,67.996619,24391680.0,0.0,0.141062,0.518791,0.659852,1.207823,1.515490,13.637390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,localhost:50003,info,Advertisement(apple > 0.9646406184803333) 09eae1c4-2297-4467-914f-5d78ad86c9ee,2024-11-30 17:36:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.656242,23986176.0,0.0,0.330613,0.823665,1.154278,2.463658,0.866342,19.490591,NaN,61.591855,24760320.0,0.0,0.122821,0.137538,0.260359,1.597563,3.840855,18.762978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6966,localhost:50003,info,Advertisement(apple > 0.3009694439421689) 68e848c3-aa65-46a7-8469-031f0cb1904b,2024-11-30 19:35:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.598151,42082304.0,0.0,0.200070,0.514997,0.715067,1.031847,1.398471,7195.916349,NaN,64.071701,77500416.0,33080.0,3528.166410,0.100027,3528.266437,0.105465,0.000283,7195.394774,NaN,64.559301,39182336.0,1.0,0.110505,0.109571,0.220075,1.05081,4.543899,3666.940526,NaN
6967,localhost:50003,info,Advertisement(apple > 0.9005249814991677) ca13b4e1-5f30-4f7b-bfa1-233d22c312c5,2024-11-30 19:35:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.596347,42082304.0,0.0,0.513543,0.510554,1.024097,1.031727,0.976470,7196.430488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6968,localhost:50003,info,Advertisement(apple > 0.019574597050384904) d05af190-75cb-492f-991e-79524e469054,2024-11-30 19:35

In [9]:
# 컬럼별 데이터 타입 확인
hop_df.dtypes

Node                    object
level                   object
msg                     object
time                    object
B1_Cpu                 float64
B1_Memory              float64
B1_QueueLength         float64
B1_QueueTime           float64
B1_ServiceTime         float64
B1_ResponseTime        float64
B1_InterArrivalTime    float64
B1_Throughput          float64
B1_m                   float64
B1_TimestampDiff       float64
B2_Cpu                 float64
B2_Memory              float64
B2_QueueLength         float64
B2_QueueTime           float64
B2_ServiceTime         float64
B2_ResponseTime        float64
B2_InterArrivalTime    float64
B2_Throughput          float64
B2_m                   float64
B2_TimestampDiff       float64
B3_Cpu                 float64
B3_Memory              float64
B3_QueueLength         float64
B3_QueueTime           float64
B3_ServiceTime         float64
B3_ResponseTime        float64
B3_InterArrivalTime    float64
B3_Throughput          float64
B3_m    

In [10]:
# hop_df의 time 컬럼의 중복되는 값과 그 개수 확인
# hop_df['time'].value_counts()
hop_df['time'].duplicated().sum()

2306

1초에 2개 이상의 행이 존재할 때\
0을 제외한 값들끼리의 평균값(또는 최댓값)을 구해서 하나의 date에는 하나의 행만 남김

In [11]:
hop_df = aggregate_rows_by_time.handle_grouped_duplicate_time(hop_df, method='mean')


In [12]:
# hop_df의 time 컬럼의 중복값 확인
hop_df['time'].duplicated().sum()

0

In [13]:
# 파일 저장
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

hop_df.to_csv(path_mergeddf + 'hop_df_agg.csv', index=False)

In [14]:
hop_df

,time,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_Cpu,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_InterArrivalTime,B3_Throughput,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,2024-11-30 17:35:54,64.282521,22675456.0,0.0,0.303324,0.000000,0.303324,0.000000,3.296801,5.818397,NaN,53.477604,23056384.0,0.0,0.311918,0.000000,0.311918,0.000000,3.205973,6.508137,NaN,71.543500,22896640.0,0.0,0.308376,0.000000,0.308376,0.000000,3.242796,7.150063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-30 17:35:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.700718,23171072.0,0.0,0.303256,0.622440,0.925696,1.280409,1.080268,9.705761,NaN,66.722926,23662592.0,0.0,0.100927,0.493589,0.594516,0.559047,1.682040,8.676234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-11-30 17:35:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.441196,23486464.0,0.0,0.329153,0.723655,1.052808,1.671994,0.949840,12.186823,NaN,69.896775,24137728.0,0.0,0.113377,0.115075,0.228452,0.971101,4.377291,11.454993,NaN,67.067395,22700032.0,0.0,0.126521,0.141435,0.267956,0.94493,3.731953,10.936167,NaN
3,2024-11-30 17:36:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.739492,23744512.0,0.0,0.326061,0.820683,1.146745,1.837630,0.872034,14.518270,NaN,67.996619,24391680.0,0.0,0.141062,0.518791,0.659852,1.207823,1.515490,13.637390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-11-30 17:36:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.656242,23986176.0,0.0,0.330613,0.823665,1.154278,2.463658,0.866342,19.490591,NaN,61.591855,24760320.0,0.0,0.122821,0.137538,0.260359,1.597563,3.840855,18.762978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4660,2024-11-30 19:35:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.598151,42082304.0,0.0,0.200070,0.514997,0.715067,1.031847,1.398471,7195.916349,NaN,64.071701,77500416.0,33080.0,3528.166410,0.100027,3528.266437,0.105465,0.000283,7195.394774,NaN,64.559301,39182336.0,1.0,0.110505,0.109571,0.220075,1.05081,4.543899,3666.940526,NaN
4661,2024-11-30 19:35:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.596347,42082304.0,0.0,0.513543,0.510554,1.024097,1.031727,0.976470,7196.430488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4662,2024-11-30 19:35:44,52.646949,41234432.0,1.0,0.308721,0.309602,0.618323,0.965343,1.617278,7191.586870,NaN,52.405883,42475520.0,6.0,4.297847,0.624734,4.922580,1.103614,0.203145,7196.256998,NaN,52.596375,42082304.0,0.0,0.701362,0.830105,1.531467,1.031671,0.652969,7197.258586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4663,2024-11-30 19:35:45,52.644325,41234432.0,1.0,0.736928,0.315029,1.051957,0.965119,0.950609,7192.835759,NaN,52.403985,42475520.0,4.0,4.917312,0.620735,5.538047,1.103524,0.180569,7198.127690,NaN,52.595444,42082304.0,0.0,0.205411,0.512855,0.718266,1.031791,1.392242,7198.633591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Tick Log 전처리

In [15]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/tickLogger/116/'
data = []
with open(path_ticklog + f'{BROKER_PORT}_tick.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
tick_df_raw = pd.DataFrame(data)
type(tick_df_raw)

pandas.core.frame.DataFrame

In [16]:
# time 컬럼의 중복값과 그 개수 확인

duplicates = tick_df_raw['time'].value_counts()

# 중복이 있는 경우만 출력 (카운트가 1보다 큰 경우)
duplicates = duplicates[duplicates > 1]

print("중복된 time 값과 그 개수:")
print(duplicates)

# 전체 중복 레코드 수 확인
total_duplicates = len(tick_df_raw) - len(tick_df_raw['time'].unique())
print(f"\n총 중복 레코드 수: {total_duplicates}")


중복된 time 값과 그 개수:
Series([], Name: time, dtype: int64)

총 중복 레코드 수: 0


In [17]:
# AverageQueueTime, AverageServiceTime, AverageThroughput, InterArrivalTime, Memory, MessageCount, QueueLength, QueueTime, ResponseTime, ServiceTime, Throughtput, TotalArrival Time 컬럼 값을 float 타입으로 변환
# 여러 컬럼 한 번에 type 변경
tick_df = tick_df_raw.astype({'AverageQueueTime': float, 
                              'AverageServiceTime': float, 
                              'AverageThroughput': float, 
                              'InterArrivalTime': float, 
                              'Memory': float, 
                              'MessageCount': float, 
                              'QueueLength': float, 
                              'QueueTime': float, 
                              'ResponseTime': float, 
                              'ServiceTime': float, 
                              'Throughput': float, 
                              'TotalArrival Time': float})


In [18]:
# 파일 저장
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

tick_df.to_csv(path_mergeddf + 'tick_df.csv', index=False)

In [19]:
tick_df.dtypes


AverageQueueTime      float64
AverageServiceTime    float64
AverageThroughput     float64
Bottleneck               bool
Cpu                   float64
InterArrivalTime      float64
Memory                float64
MessageCount          float64
QueueLength           float64
QueueTime             float64
ResponseTime          float64
ServiceTime           float64
Throughput            float64
TotalArrival Time     float64
level                  object
msg                    object
time                   object
dtype: object

In [20]:
tick_df

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,Cpu,InterArrivalTime,Memory,MessageCount,QueueLength,QueueTime,ResponseTime,ServiceTime,Throughput,TotalArrival Time,level,msg,time
0,0.000000,0.000000,0.000000,False,48.486388,0.000000,21032960.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:47
1,0.000000,0.000000,0.000000,False,56.598284,0.000000,22704128.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:48
2,0.000000,0.000000,0.000000,False,61.668857,0.000000,22716416.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:49
3,0.000000,0.000000,0.000000,False,63.839714,0.000000,22724608.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:50
4,0.000000,0.000000,0.000000,False,67.707828,0.000000,22732800.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,45.141718,0.814160,0.021760,False,52.597383,1.031655,42082304.0,6966.0,0.0,0.201589,0.716586,0.514997,1.395506,7186.512139,info,Performance Metrics,2024-11-30 19:35:41
7195,45.135267,0.814160,0.021763,False,52.598102,1.031727,42082304.0,6968.0,1.0,0.200070,0.715067,0.514997,1.398471,7189.075258,info,Performance Metrics,2024-11-30 19:35:42
7196,45.128863,0.814116,0.021766,False,52.596225,1.031671,42082304.0,6969.0,1.0,0.513543,1.024097,0.510554,0.976470,7189.715536,info,Performance Metrics,2024-11-30 19:35:43
7197,45.122488,0.814075,0.021769,False,52.595472,1.031671,42082304.0,6969.0,0.0,0.701362,1.214218,0.512855,0.823576,7189.715536,info,Performance Metrics,2024-11-30 19:35:44


In [21]:
merged_df = merge_hop_tick.merge_dataframes(hop_df, tick_df, BROKER_PORT)

In [22]:
merged_df

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,B3_Cpu,B3_InterArrivalTime,B3_Memory,MessageCount,B3_QueueLength,B3_QueueTime,B3_ResponseTime,B3_ServiceTime,B3_Throughput,TotalArrival Time,level,msg,time,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,0.000000,0.000000,0.000000,False,48.486388,0.000000,21032960.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,False,56.598284,0.000000,22704128.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0.000000,False,61.668857,0.000000,22716416.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,0.000000,False,63.839714,0.000000,22724608.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,0.000000,0.000000,False,67.707828,0.000000,22732800.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,45.141718,0.814160,0.021760,False,52.597383,1.031655,42082304.0,6966.0,0.0,0.201589,0.716586,0.514997,1.395506,7186.512139,info,Performance Metrics,2024-11-30 19:35:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7195,45.135267,0.814160,0.021763,False,52.598151,1.031847,42082304.0,6968.0,0.0,0.200070,0.715067,0.514997,1.398471,7189.075258,info,Performance Metrics,2024-11-30 19:35:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7195.916349,NaN,64.071701,77500416.0,33080.0,3528.16641,0.100027,3528.266437,0.105465,0.000283,7195.394774,NaN,64.559301,39182336.0,1.0,0.110505,0.109571,0.220075,1.05081,4.543899,3666.940526,NaN
7196,45.128863,0.814116,0.021766,False,52.596347,1.031727,42082304.0,6969.0,0.0,0.513543,1.024097,0.510554,0.976470,7189.715536,info,Performance Metrics,2024-11-30 19:35:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7196.430488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7197,45.122488,0.814075,0.021769,False,52.596375,1.031671,42082304.0,6969.0,0.0,0.701362,1.531467,0.830105,0.652969,7189.715536,info,Performance Metrics,2024-11-30 19:35:44,52.646949,41234432.0,

In [23]:
merged_df['Bottleneck'] = merged_df['Bottleneck'].astype('float64')
merged_df.columns

Index(['AverageQueueTime', 'AverageServiceTime', 'AverageThroughput',
       'Bottleneck', 'B3_Cpu', 'B3_InterArrivalTime', 'B3_Memory',
       'MessageCount', 'B3_QueueLength', 'B3_QueueTime', 'B3_ResponseTime',
       'B3_ServiceTime', 'B3_Throughput', 'TotalArrival Time', 'level', 'msg',
       'time', 'B1_Cpu', 'B1_Memory', 'B1_QueueLength', 'B1_QueueTime',
       'B1_ServiceTime', 'B1_ResponseTime', 'B1_InterArrivalTime',
       'B1_Throughput', 'B1_m', 'B1_TimestampDiff', 'B2_Cpu', 'B2_Memory',
       'B2_QueueLength', 'B2_QueueTime', 'B2_ServiceTime', 'B2_ResponseTime',
       'B2_InterArrivalTime', 'B2_Throughput', 'B2_m', 'B2_TimestampDiff',
       'B3_m', 'B3_TimestampDiff', 'B4_Cpu', 'B4_Memory', 'B4_QueueLength',
       'B4_QueueTime', 'B4_ServiceTime', 'B4_ResponseTime',
       'B4_InterArrivalTime', 'B4_Throughput', 'B4_m', 'B4_TimestampDiff',
       'B5_Cpu', 'B5_Memory', 'B5_QueueLength', 'B5_QueueTime',
       'B5_ServiceTime', 'B5_ResponseTime', 'B5_InterArrivalTime',

In [24]:
# 학습에 불필요한 컬럼 제거
# 불필요한 컬럼 리스트
drop_columns = [
    'level',
    'msg',
    # 'Node',
    # 'TotalArrivalTime',
    'AverageQueueTime',
    'AverageServiceTime',
    'AverageThroughput',
    
    'MessageCount',
    # 'Timestamp'
]

# 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

In [25]:
merged_df = merged_df.rename(columns={'time': 'date'})

In [26]:
merged_df

,Bottleneck,B3_Cpu,B3_InterArrivalTime,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ResponseTime,B3_ServiceTime,B3_Throughput,TotalArrival Time,date,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,0.0,48.486388,0.000000,21032960.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,56.598284,0.000000,22704128.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,61.668857,0.000000,22716416.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,63.839714,0.000000,22724608.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,67.707828,0.000000,22732800.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,0.0,52.597383,1.031655,42082304.0,0.0,0.201589,0.716586,0.514997,1.395506,7186.512139,2024-11-30 19:35:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7195,0.0,52.598151,1.031847,42082304.0,0.0,0.200070,0.715067,0.514997,1.398471,7189.075258,2024-11-30 19:35:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7195.916349,NaN,64.071701,77500416.0,33080.0,3528.16641,0.100027,3528.266437,0.105465,0.000283,7195.394774,NaN,64.559301,39182336.0,1.0,0.110505,0.109571,0.220075,1.05081,4.543899,3666.940526,NaN
7196,0.0,52.596347,1.031727,42082304.0,0.0,0.513543,1.024097,0.510554,0.976470,7189.715536,2024-11-30 19:35:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7196.430488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7197,0.0,52.596375,1.031671,42082304.0,0.0,0.701362,1.531467,0.830105,0.652969,7189.715536,2024-11-30 19:35:44,52.646949,41234432.0,1.0,0.308721,0.309602,0.618323,0.965343,1.617278,7191.586870,NaN,52.405883,42475520.0,6.0,4.297847,0.624734,4.922580,1.103614,0.203145,7196.256998,NaN,7197.258586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# # 모든 값이 NA인 열들을 확인 후 DataFrame에서 삭제
# print(merged_df.columns[merged_df.isna().all()].tolist())
# merged_df.drop(columns=merged_df.columns[merged_df.isna().all()].tolist(), inplace=True)


In [28]:
# merge_df 의 컬럼 출력
merged_df.columns

Index(['Bottleneck', 'B3_Cpu', 'B3_InterArrivalTime', 'B3_Memory',
       'B3_QueueLength', 'B3_QueueTime', 'B3_ResponseTime', 'B3_ServiceTime',
       'B3_Throughput', 'TotalArrival Time', 'date', 'B1_Cpu', 'B1_Memory',
       'B1_QueueLength', 'B1_QueueTime', 'B1_ServiceTime', 'B1_ResponseTime',
       'B1_InterArrivalTime', 'B1_Throughput', 'B1_m', 'B1_TimestampDiff',
       'B2_Cpu', 'B2_Memory', 'B2_QueueLength', 'B2_QueueTime',
       'B2_ServiceTime', 'B2_ResponseTime', 'B2_InterArrivalTime',
       'B2_Throughput', 'B2_m', 'B2_TimestampDiff', 'B3_m', 'B3_TimestampDiff',
       'B4_Cpu', 'B4_Memory', 'B4_QueueLength', 'B4_QueueTime',
       'B4_ServiceTime', 'B4_ResponseTime', 'B4_InterArrivalTime',
       'B4_Throughput', 'B4_m', 'B4_TimestampDiff', 'B5_Cpu', 'B5_Memory',
       'B5_QueueLength', 'B5_QueueTime', 'B5_ServiceTime', 'B5_ResponseTime',
       'B5_InterArrivalTime', 'B5_Throughput', 'B5_m', 'B5_TimestampDiff'],
      dtype='object')

In [29]:
new_order = ['date', 'Bottleneck',
            #  'HopCount', 
            #  'B1_Timestamp', 'B1_BrokerId',
             'B1_Cpu', 'B1_Memory', 'B1_InterArrivalTime', 'B1_QueueLength', 'B1_QueueTime', 'B1_ServiceTime', 'B1_ResponseTime', 'B1_Throughput',
            #  'B2_Timestamp',  'B2_BrokerId',
             'B2_Cpu', 'B2_Memory', 'B2_InterArrivalTime', 'B2_QueueLength',  'B2_QueueTime', 'B2_ServiceTime', 'B2_ResponseTime', 'B2_Throughput',
            #  'B3_Timestamp', 'B3_BrokerId',
             'B3_Cpu', 'B3_Memory', 'B3_InterArrivalTime', 'B3_QueueLength', 'B3_QueueTime', 'B3_ServiceTime', 'B3_ResponseTime', 'B3_Throughput',
            #  'B4_Timestamp', 'B4_BrokerId',
             'B4_Cpu', 'B4_Memory', 'B4_InterArrivalTime', 'B4_QueueLength', 'B4_QueueTime', 'B4_ServiceTime', 'B4_ResponseTime', 'B4_Throughput',
            #  'B5_Timestamp', 'B5_BrokerId',
             'B5_Cpu', 'B5_Memory', 'B5_InterArrivalTime', 'B5_QueueLength', 'B5_QueueTime', 'B5_ServiceTime', 'B5_ResponseTime', 'B5_Throughput',]

# 새로운 순서로 데이터프레임을 재정렬합니다
merged_df = merged_df[new_order]

In [30]:
drop_columns = [ 
    'Bottleneck',
#     'BrokerId_0',
#     'BrokerId_1',
#     'BrokerId_2'
]

# # 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

c:\Users\SJSJ\anaconda3\envs\informer2020\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [31]:
# return all unique values of merged_df columns
for col in merged_df.columns:
    print(col)
    print(merged_df[col].unique())
    print()


date
['2024-11-30 17:35:47' '2024-11-30 17:35:48' '2024-11-30 17:35:49' ...
 '2024-11-30 19:35:43' '2024-11-30 19:35:44' '2024-11-30 19:35:45']

B1_Cpu
[       nan 64.282521  60.003969  58.096223  56.068958  54.728934
 52.422291  46.452392  45.976253  45.990919  46.002563  46.340821
 46.668221  46.828846  46.839007  46.638731  46.676355  46.371894
 46.592231  46.671873  46.814189  46.858924  46.924777  46.911714
 46.865506  46.984085  47.254136  47.248156  47.207772  47.250275
 47.259871  47.349832  47.601573  47.545331  47.626647  47.664565
 47.836848  47.881857  47.875106  47.73192   47.903681  47.94466
 47.96978   47.858784  47.913908  48.198242  48.197993  48.138794
 48.042742  47.885712  47.958348  48.010035  48.020952  48.031129
 48.01242   48.139556  48.06253   48.0794    48.112583  48.115814
 48.144258  48.213374  48.336718  48.36062   48.496937  48.560497
 48.782119  48.804731  48.850404  48.844474  48.865244  48.882316
 48.858308  48.931832  48.921148  48.959472  48.981862  4

In [32]:
# NaN 값을 0으로 대체
merged_df.fillna(0, inplace=True)

c:\Users\SJSJ\anaconda3\envs\informer2020\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [33]:
merged_df

,date,B1_Cpu,B1_Memory,B1_InterArrivalTime,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_Throughput,B2_Cpu,B2_Memory,B2_InterArrivalTime,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_Throughput,B3_Cpu,B3_Memory,B3_InterArrivalTime,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_Throughput,B4_Cpu,B4_Memory,B4_InterArrivalTime,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_Throughput,B5_Cpu,B5_Memory,B5_InterArrivalTime,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_Throughput
0,2024-11-30 17:35:47,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,48.486388,21032960.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
1,2024-11-30 17:35:48,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,56.598284,22704128.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
2,2024-11-30 17:35:49,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,61.668857,22716416.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
3,2024-11-30 17:35:50,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,63.839714,22724608.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
4,2024-11-30 17:35:51,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,67.707828,22732800.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,2024-11-30 19:35:41,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,52.597383,42082304.0,1.031655,0.0,0.201589,0.514997,0.716586,1.395506,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
7195,2024-11-30 19:35:42,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,52.598151,42082304.0,1.031847,0.0,0.200070,0.514997,0.715067,1.398471,64.071701,77500416.0,0.105465,33080.0,3528.16641,0.100027,3528.266437,0.000283,64.559301,39182336.0,1.05081,1.0,0.110505,0.109571,0.220075,4.543899
7196,2024-11-30 19:35:43,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,52.596347,42082304.0,1.031727,0.0,0.513543,0.510554,1.024097,0.976470,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
7197,2024-11-30 19:35:44,52.646949,41234432.0,0.965343,1.0,0.308721,0.309602,0.618323,1.617278,52.405883,42475520.0,1.103614,6.0,4.297847,0.624734,4.922580,0.203145,52.596375,42082304.0,1.031671,0.0,0.701362,0.830105,1.531467,0.652969,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000


In [34]:
# # 각 컬럼별로 unique 값이 하나인 컬럼 삭제
# for col in merged_df.columns:
#     if len(merged_df[col].unique()) == 1:
#         print(col)
#         merged_df = merged_df.drop(col, axis=1)

In [35]:
# save to csv
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

merged_df.to_csv(path_mergeddf +'merged_df.csv', date_format='%Y-%m-%d %H:%M:%S', index=False)


In [36]:
merged_df.dtypes

date                    object
B1_Cpu                 float64
B1_Memory              float64
B1_InterArrivalTime    float64
B1_QueueLength         float64
B1_QueueTime           float64
B1_ServiceTime         float64
B1_ResponseTime        float64
B1_Throughput          float64
B2_Cpu                 float64
B2_Memory              float64
B2_InterArrivalTime    float64
B2_QueueLength         float64
B2_QueueTime           float64
B2_ServiceTime         float64
B2_ResponseTime        float64
B2_Throughput          float64
B3_Cpu                 float64
B3_Memory              float64
B3_InterArrivalTime    float64
B3_QueueLength         float64
B3_QueueTime           float64
B3_ServiceTime         float64
B3_ResponseTime        float64
B3_Throughput          float64
B4_Cpu                 float64
B4_Memory              float64
B4_InterArrivalTime    float64
B4_QueueLength         float64
B4_QueueTime           float64
B4_ServiceTime         float64
B4_ResponseTime        float64
B4_Throu

In [37]:
merged_df.columns

Index(['date', 'B1_Cpu', 'B1_Memory', 'B1_InterArrivalTime', 'B1_QueueLength',
       'B1_QueueTime', 'B1_ServiceTime', 'B1_ResponseTime', 'B1_Throughput',
       'B2_Cpu', 'B2_Memory', 'B2_InterArrivalTime', 'B2_QueueLength',
       'B2_QueueTime', 'B2_ServiceTime', 'B2_ResponseTime', 'B2_Throughput',
       'B3_Cpu', 'B3_Memory', 'B3_InterArrivalTime', 'B3_QueueLength',
       'B3_QueueTime', 'B3_ServiceTime', 'B3_ResponseTime', 'B3_Throughput',
       'B4_Cpu', 'B4_Memory', 'B4_InterArrivalTime', 'B4_QueueLength',
       'B4_QueueTime', 'B4_ServiceTime', 'B4_ResponseTime', 'B4_Throughput',
       'B5_Cpu', 'B5_Memory', 'B5_InterArrivalTime', 'B5_QueueLength',
       'B5_QueueTime', 'B5_ServiceTime', 'B5_ResponseTime', 'B5_Throughput'],
      dtype='object')

In [38]:
# date 컬럼의 중복값과 그 개수 확인

duplicates = merged_df['date'].value_counts()

# 중복이 있는 경우만 출력 (카운트가 1보다 큰 경우)
duplicates = duplicates[duplicates > 1]

print("중복된 date 값과 그 개수:")
print(duplicates)

# 전체 중복 레코드 수 확인
total_duplicates = len(merged_df) - len(merged_df['date'].unique())
print(f"\n총 중복 레코드 수: {total_duplicates}")


중복된 date 값과 그 개수:
Series([], Name: date, dtype: int64)

총 중복 레코드 수: 0


---

## Temporal-Spatio dataset 구축

In [39]:
# # 사용할 feature 선택

# # 해당 리스트에 있는 문자열로 끝나는 컬럼만 선택
# feature_list = ['Cpu', 'Memory', 'InterArrivalTime', 'QueueLength', 'QueueTime', 'ServiceTime', 'ResponseTime', 'Throughput']

# # feature_list에 있는 문자열로 끝나는 컬럼만 선택
# selected_features = merged_df.filter(regex='|'.join(feature_list))

# selected_features
